In [1]:
import numpy as np 
import cv2
import os

In [2]:
def reverseColumn(img):
    return img[::-1 , :]
    


In [3]:
def reverseRow(img):
    return img[: ,::-1]

In [4]:
def getTranspose(img):
    return img.swapaxes(0, 1)

In [5]:
def reverseEntireMatrix(img):
    return img[::-1 , ::-1]

In [6]:
def RotateImageBy90(img):
    img = reverseColumn(img)
    img = getTranspose(img)
    return img

In [7]:
def RotateImageBy180(img):
    img = reverseRow(img)
    img = reverseEntireMatrix(img)

    return img

In [8]:
def RotateImageBy270(img):
    img = reverseRow(img)
    img = getTranspose(img)
    return img

In [9]:
def FlipImageHorizontally(img):
    img = reverseRow(img)
    return img

In [10]:
def FlipImageVertically(img):
    img =  reverseEntireMatrix(img)
    return img

In [11]:
def getImageSize(img):
    height, width = img.shape[:2]
    return height , width

In [12]:
def ScaleImage(img , newHeight , newWidth):
    img = cv2.resize(img, (newWidth, newHeight))
    return img

In [13]:
def IncreaseBrightness(img , value):
    img = img + value
    img = np.clip(img, 0, 255)
    return img


In [14]:
def DecreaseBrightness(img , value):
    img = img - value
    img = np.clip(img, 0, 255)
    return img

In [15]:
def ChangeContrast(img , alpha): # if alpha > 1 -> increase contrast , if alpha < 1 -> decrease contrast 
    img = alpha * img 
    img = np.clip(img , 0 , 255)
    return img

In [16]:
def saveImage(img , path , num):
    folder_name = os.path.basename(path)
    cv2.imwrite(f"{path}\\{folder_name}_augImage_{num}.jpg", img)

In [17]:
def ApplyAugumentationTechniques(images , folderPath):
    n = -1 
    cnt = 0
    H, W, C = 256, 256, 3  
    augmented_image = np.zeros((H, W, C), dtype=np.uint8)
    while cnt < 164:
      for img in images:
        if cnt >= 164:
          return
        cnt += 1
        n = (n + 1) % 10
        match n:
            case 0:
              augmented_image = RotateImageBy90(img)  
            case 1:
              augmented_image = RotateImageBy180(img) 
            case 2:
              augmented_image = RotateImageBy270(img) 
            case 3:
              augmented_image = FlipImageHorizontally(img)
            case 4:
              augmented_image = FlipImageVertically(img)
            case 5:
              h, w = img.shape[:2]
              nh = np.random.randint(int(h*0.5), int(h*2.0))  
              nw = np.random.randint(int(w*0.5), int(w*2.0))
              augmented_image = ScaleImage(img , nh , nw)
            case 6:
              value = np.random.randint(1 , 100)
              augmented_image = IncreaseBrightness(img , value)
            case 7:
              value = np.random.randint(1 , 100)
              augmented_image = DecreaseBrightness(img , value)
            case 8:
              alpha = np.random.uniform(1.1, 2.5) 
              augmented_image = ChangeContrast(img, alpha)
            case 9:
              alpha = np.random.rand()
              augmented_image = ChangeContrast(img , alpha)
        saveImage(augmented_image , folderPath , cnt) 
        print(f"augumented_image {cnt} saved successfuly!" )
        

In [18]:
folderPaths = [
    # r"C:\Users\Fatma Ibrahim\Desktop\Dataset\cardboard",
    # r"C:\Users\Fatma Ibrahim\Desktop\Dataset\glass",
    # r"C:\Users\Fatma Ibrahim\Desktop\Dataset\metal",
    # r"C:\Users\Fatma Ibrahim\Desktop\Dataset\paper",
    # r"C:\Users\Fatma Ibrahim\Desktop\Dataset\plastic",
    # r"C:\Users\Fatma Ibrahim\Desktop\Dataset\trash" 
    r"E:\My_Folders\FCAI_Courses\4th_Year_Courses\first_term\ML\project\Material_Stream_Identification_System\dataset\cardboard",
    r"E:\My_Folders\FCAI_Courses\4th_Year_Courses\first_term\ML\project\Material_Stream_Identification_System\dataset\glass",
    r"E:\My_Folders\FCAI_Courses\4th_Year_Courses\first_term\ML\project\Material_Stream_Identification_System\dataset\metal",
    r"E:\My_Folders\FCAI_Courses\4th_Year_Courses\first_term\ML\project\Material_Stream_Identification_System\dataset\paper",
    r"E:\My_Folders\FCAI_Courses\4th_Year_Courses\first_term\ML\project\Material_Stream_Identification_System\dataset\plastic",
    r"E:\My_Folders\FCAI_Courses\4th_Year_Courses\first_term\ML\project\Material_Stream_Identification_System\dataset\trash"   
      ]
for folder in folderPaths:
    images = []
    for filename in os.listdir(folder):
        file_path = os.path.join(folder,filename)
        if os.path.isfile(file_path) and filename.lower().endswith('.jpg'):
          img = cv2.imread(file_path)
          if img is None:
            os.remove(file_path)
            print(f"{file_path} is removed , The image is corrupted and won't open ")
            continue 
          images.append(img)
    ApplyAugumentationTechniques(images , folder)
    
            


E:\My_Folders\FCAI_Courses\4th_Year_Courses\first_term\ML\project\Material_Stream_Identification_System\dataset\cardboard\2ec9d19b-8027-4c77-a13f-5eee033b9868.jpg is removed , The image is corrupted and won't open 
E:\My_Folders\FCAI_Courses\4th_Year_Courses\first_term\ML\project\Material_Stream_Identification_System\dataset\cardboard\31381a44-38d6-4a44-9384-7690727801bc.jpg is removed , The image is corrupted and won't open 
E:\My_Folders\FCAI_Courses\4th_Year_Courses\first_term\ML\project\Material_Stream_Identification_System\dataset\cardboard\345bdb67-4190-4235-a16f-b60c1556a28d.jpg is removed , The image is corrupted and won't open 
E:\My_Folders\FCAI_Courses\4th_Year_Courses\first_term\ML\project\Material_Stream_Identification_System\dataset\cardboard\38b3e4da-738c-4694-a946-55101b25ad53.jpg is removed , The image is corrupted and won't open 
E:\My_Folders\FCAI_Courses\4th_Year_Courses\first_term\ML\project\Material_Stream_Identification_System\dataset\cardboard\4840d678-7af4-4a2d